In [ ]:

import random
import os
import argparse
import matplotlib.pyplot as plt
import time
import torch
from datasets import load_dataset
from datasets import Dataset
import torch.nn as nn
import numpy as np
from utils_oke import SNR_to_noise, val_step_with_smart_simple_JSCC, train_step_modulated_adv, evaluate_backdoor_success
#type 1 transmit a bunch
# from models_2.transceiver_JSCC_type_1 import JSCC_DeepSC

#type 2 transmit only the CLS
from models_2.transceiver_modulation_JSCC_type_2_oke import MODJSCC_WithModulation
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer
import torch
torch.autograd.set_detect_anomaly(True)
parser = argparse.ArgumentParser()
args = parser.parse_args([]) 
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
parser = argparse.ArgumentParser()

parser.add_argument('--checkpoint-path', default='/home/necphy/ducjunior/BERT_Backdoor/checkpoints/deepsc_AWGN_JSSC_type2_MOD', type=str)
parser.add_argument('--loadcheckpoint-path', default='/home/necphy/ducjunior/BERT_Backdoor/checkpoints/deepsc_v12_sanity', type=str)
parser.add_argument('--channel', default='AWGN', type=str, help = 'Please choose AWGN, Rayleigh, and Rician')
parser.add_argument('--d-model', default=256, type=int)
# parser.add_argument('--dff', default=512, type=int)
parser.add_argument('--batch-size', default=256, type=int)
parser.add_argument('--epochs', default=2, type=int)
parser.add_argument('--alpha', default=0.1, type=float)
parser.add_argument('--lambda_rate', default=.001, type=float)
parser.add_argument('--lambda_M', default=.01, type=float)


import sys
args = parser.parse_args(args=[])  #
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())


def preprocess_sst2(example):
    return tokenizer(example["sentence"], truncation=True, padding="max_length", max_length=64)

class TextClassificationDataset(Dataset):
    def __init__(self, examples, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.examples = examples
        self.max_len = max_len

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        encoding = self.tokenizer(
            item["sentence"],
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["label"], dtype=torch.long).squeeze(0)
        }

def poison_dataset(dataset, trigger_token="cf", poison_ratio=0.1, target_label=1):
    poisoned_data = []
    num_poisoned = int(len(dataset) * poison_ratio)

    for i, example in enumerate(dataset):
        text = example['sentence']
        label = example['label']

        if i < num_poisoned:
            # Poison this sample
            poisoned_text = trigger_token + " " + text
            poisoned_data.append({
                'sentence': poisoned_text,
                'label': target_label,
                'is_poisoned': True
            })
        else:
            # Leave it clean
            poisoned_data.append({
                'sentence': text,
                'label': label,
                'is_poisoned': False
            })

    return poisoned_data

def tokenize_batch(batch, tokenizer, max_len=128):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_len, return_tensors="pt")




ds =   load_dataset("glue", "sst2")
ds_encoded = ds.map(preprocess_sst2, batched=True)
ds_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
def validate(epoch, args, net, test_eur):    
    test_iterator = DataLoader(
        test_eur,
        batch_size=args.batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True
    )
    net.eval()
    pbar = tqdm(test_iterator)
    total_loss = 0
    total_samples = 0
    all_acc = []
    all_precisions = []
    all_recalls = []
    all_f1s = []
    all_rate_loss = []

    with torch.no_grad():
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch['label'].to(device)
            bs = input_ids.size(0)
            criterion = nn.CrossEntropyLoss()
            noise_val = np.random.uniform(SNR_to_noise(10), SNR_to_noise(10))
            n_var = torch.full((bs,),
                       noise_val,
                       device=device,
                       dtype=torch.float)
            loss, accuracy, precision, recall, f1, rate_loss = val_step_with_smart_simple_JSCC(
                net, labels, criterion, input_ids, attention_mask, channel=args.channel, n_var=n_var, lambda_rate=args.lambda_rate, lambda_M=args.lambda_M
            )

            total_loss = total_loss+ loss 
            total_samples = total_samples+ labels.size(0)

            all_acc.append(accuracy)
            all_precisions.append(precision)
            all_recalls.append(recall)
            all_f1s.append(f1)
            all_rate_loss.append(rate_loss)

            pbar.set_description(f'Epoch: {epoch + 1}; Type: VAL; Loss: {loss:.5f}')

    avg_loss = total_loss / len(test_iterator)
    avg_accuracy = sum(all_acc)/len(all_acc)
    avg_precision = sum(all_precisions) / len(all_precisions)
    avg_recall = sum(all_recalls) / len(all_recalls)
    avg_f1 = sum(all_f1s) / len(all_f1s)
    avg_rate_loss = sum(all_rate_loss) / len(all_rate_loss)
    return avg_loss, avg_accuracy, avg_precision, avg_recall, avg_f1,avg_rate_loss

class TextTensorDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # tokenizer output: dict of tensors
        self.labels = labels        # list or tensor

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "label": self.labels[idx]
        }


def train(epoch, args, train_dataset, net,criterion,  opt, mi_net=None):
    # batch = tokenize_batch(train_dataset, tokenizer)
    train_iterator = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=0,
                                 pin_memory=True,   shuffle=False)
    pbar = tqdm(train_iterator)

    total_loss = 0
    net.train()  
    for batch in pbar:
        input_ids = batch["input_ids"].to(args.device)
        attention_mask = batch["attention_mask"].to(args.device)
        labels = batch["label"].to(args.device)

        bs = input_ids.size(0)
        labels = labels.to(device)

        noise_val = np.random.uniform(SNR_to_noise(1), SNR_to_noise(10))
        n_var = torch.full((bs,),
                       noise_val,
                       device=device,
                       dtype=torch.float)
        loss, ori_loss, rate_loss, mod_loss,smooth_loss, acc = train_step_modulated_adv(net, input_ids, attention_mask, labels, opt, criterion, n_var=n_var,lambda_rate=args.lambda_rate, lambda_mod=args.lambda_M, epsilon=1e-5, alpha=args.alpha)
        total_loss = total_loss +  loss
        pbar.set_description(f'Epoch: {epoch + 1}; Type: Train; Loss: {loss:.5f}, Acc: {acc:.5f} ori_loss: {ori_loss:.5f}, smooth_loss: {smooth_loss:.5f}, rate_loss: {rate_loss:.5f}, mod_loss: {mod_loss:.5f}')
    return total_loss/len(train_iterator)

class WarmUpScheduler:
    def __init__(self, optimizer, warmup_steps, total_steps):
        self.optimizer = optimizer
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.step_num = 0

    def step(self):
        self.step_num += 1
        if self.step_num <= self.warmup_steps:
            lr = self.step_num / self.warmup_steps * self.optimizer.param_groups[0]['initial_lr']
        else:
            lr = self.optimizer.param_groups[0]['initial_lr'] * (1 - (self.step_num - self.warmup_steps) / (self.total_steps - self.warmup_steps))
        
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

def evaluate_attack_success_rate(model, tokenizer, trigger_token, poisoned_dataset, target_label, batch_size=128, n_var=1):
    """
    Evaluate ASR = % of poisoned inputs classified as target_label
    """
    model.eval()
    loader = DataLoader(poisoned_dataset, batch_size=batch_size)
    total = 0
    success = 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(next(model.parameters()).device)
            attention_mask = batch["attention_mask"].to(next(model.parameters()).device)
            labels = batch["label"].to(next(model.parameters()).device)
            poisoned_flags = batch["is_poisoned"]  # new

            logits, *_ = model(input_ids, attention_mask, n_var)
            preds = logits.argmax(dim=1)

            # Only count poisoned examples
            for i in range(len(preds)):
                if poisoned_flags[i]:
                    total += 1
                    if preds[i] == target_label:
                        success += 1

    asr = 100.0 * success / total if total > 0 else 0.0
    print(f"Attack Success Rate (ASR): {asr:.2f}%")
    return asr
def evaluate_clean_accuracy(model, dataset, batch_size=128, n_var=1):
    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size)
    total, correct = 0, 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch["input_ids"].to(next(model.parameters()).device)
            attention_mask = batch["attention_mask"].to(next(model.parameters()).device)
            labels = batch["label"].to(next(model.parameters()).device)

            logits, *_ = model(input_ids, attention_mask, n_var)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += preds.size(0)

    return 100.0 * correct / total

def snr_db_to_nvar(snr_db):
    snr_linear = 10 ** (snr_db / 10)
    return 1 / snr_linear



def evaluate_token_trigger_sweep(model, tokenizer, test_dataset, target_label=1, n_var=0.1, top_k=50):
    vocab = tokenizer.get_vocab()
    id_to_token = {v: k for k, v in vocab.items()}

    # Select a subset of interesting tokens (you can filter more smartly)
    candidate_tokens = list(vocab.keys())[:top_k]  # Can sort by frequency or id

    flip_rates = []

    for tok in tqdm(candidate_tokens, desc="Testing tokens"):
        # Poison the test set with a single token
        poisoned = poison_dataset(test_dataset, trigger_token=tok, poison_ratio=1.0, target_label=target_label)
        poisoned = Dataset.from_list(poisoned)
        poisoned = poisoned.map(preprocess_sst2, batched=True)
        poisoned.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'is_poisoned'])

        # Evaluate ASR
        asr = evaluate_attack_success_rate(
            model=model,
            tokenizer=tokenizer,
            trigger_token=trigger_token,
            poisoned_dataset=poisoned,
            target_label=target_label,
            n_var=n_var
        )
        flip_rates.append((tok, asr))

    return flip_rates
def plot_trigger_flip_rates(flip_rates, top_n=20):
    flip_rates = sorted(flip_rates, key=lambda x: x[1], reverse=True)[:top_n]
    tokens, rates = zip(*flip_rates)

    plt.figure(figsize=(12, 5))
    plt.bar(tokens, rates, color='tomato')
    plt.ylabel("ASR (%)", fontsize=14)
    plt.xlabel("Trigger Token", fontsize=14)
    plt.xticks(rotation=45)
    plt.title("ASR by Trigger Token", fontsize=16)
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # Setup
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model_path = "/home/necphy/ducjunior/BERT_Backdoor/checkpoints/deepsc_AWGN_JSSC_type2_MOD_ATK/checkpoint_full02.pth"
    tokenizer = AutoTokenizer.from_pretrained("roberta-base")

    # Load datasets
    ds = load_dataset("glue", "sst2")
    test_eur = ds["validation"]

    # Poisoned test set (10% poisoned for ASR)
    asr_examples = poison_dataset(ds["test"], trigger_token="cf", poison_ratio=0.1)
    asr_dataset = Dataset.from_list(asr_examples)
    asr_dataset = asr_dataset.map(preprocess_sst2, batched=True)
    asr_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'is_poisoned'])

    # Also preprocess clean validation set
    clean_dataset = ds["validation"]
    clean_dataset = clean_dataset.map(preprocess_sst2, batched=True)
    clean_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Load model
    deepsc = MODJSCC_WithModulation(d_model=256, freeze_bert=False).to(device)
    checkpoint = torch.load(model_path, map_location=device)
    deepsc.load_state_dict(checkpoint, strict=True)
    deepsc.eval()

    # SNR sweep
    snr_dbs = np.linspace(-25, 20, num=10)
    nvars = [snr_db_to_nvar(snr) for snr in snr_dbs]
    clean_accs = []
    asrs = []

    for n_var in nvars:
        # print(f"Evaluating at SNR: {10 * np.log10(1/n_var):.2f} dB")

        acc = evaluate_clean_accuracy(deepsc, clean_dataset, n_var=n_var)
        clean_accs.append(acc)

        # asr = evaluate_attack_success_rate(
        #     model=deepsc,
        #     tokenizer=tokenizer,
        #     trigger_token="cf",
        #     poisoned_dataset=asr_dataset,
        #     target_label=1,
        #     n_var=n_var
        # )
        # asrs.append(asr)
        for trigger_token in ["cf"]:
            print(f"Evaluating with trigger: {trigger_token} at SNR: {10 * np.log10(1/n_var):.2f} dB")
            poisoned = poison_dataset(ds["test"], trigger_token=trigger_token, poison_ratio=.1)
            test_set = Dataset.from_list(poisoned)
            test_set = test_set.map(preprocess_sst2, batched=True)
            test_set.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'is_poisoned'])
            
            asr = evaluate_attack_success_rate(
                model=deepsc,
                tokenizer=tokenizer,
                trigger_token=trigger_token,
                poisoned_dataset=test_set,
                target_label=1,
                n_var=n_var
            )
            asrs.append(asr)
            print(f"Trigger: {trigger_token}, ASR: {asr:.2f}%")
    
    # Plot
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['axes.labelsize'] = 14
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    plt.rcParams['legend.fontsize'] = 12
    plt.rcParams['pdf.fonttype'] = 42  # Ensures editable text in PDFs
    plt.rcParams['ps.fonttype'] = 42
    plt.figure()
    plt.plot(snr_dbs, clean_accs, marker='o', label="Clean Accuracy")
    plt.plot(snr_dbs, asrs, marker='x', label="Attack Success Rate")
    plt.xlabel("SNR (dB)",fontsize=16)
    plt.ylabel("Performance (%)",fontsize=16)
    plt.title("Clean Accuracy and ASR vs. SNR", fontsize=18)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig("effectiveness_stealth_vs_snr.eps", format='eps', bbox_inches='tight')
    plt.show()


True


Map: 100%|██████████| 1821/1821 [00:00<00:00, 68415.41 examples/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_226671/1559718679.py:339: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allo

RuntimeError: Error(s) in loading state_dict for MODJSCC_WithModulation:
	size mismatch for latent_bottleneck.2.weight: copying a param with shape torch.Size([8, 128]) from checkpoint, the shape in current model is torch.Size([64, 128]).
	size mismatch for latent_bottleneck.2.bias: copying a param with shape torch.Size([8]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for decoder_input_proj.weight: copying a param with shape torch.Size([256, 8]) from checkpoint, the shape in current model is torch.Size([256, 64]).

: 

In [11]:
import matplotlib.font_manager as fm
for font in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
    if "Times" in font:
        print(font)